In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install

In [ ]:
!pip install -q evaluate
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q transformers accelerate

# !pip install -q git+https://github.com/zphang/transformers@c3dc391
# !pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
import pandas as pd
import os
import concurrent.futures
from tqdm import tqdm
import time
import evaluate
import torch

from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-87rsme9hqhhb --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=

In [ ]:

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")


model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    # torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
)

model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

# model = LLaMAForCausalLM.from_pretrained(
#     "chavinlo/alpaca-native",
#     # torch_dtype=torch.float16,
#     load_in_8bit=True,
#     device_map="auto",
# )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
# # Alpaca
# generation_config = GenerationConfig(
#     temperature=0.1,
#     top_p=0.75,
#     num_beams=4,
# )

# Llama
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(sentence, question, input=None):
    instruction = """
    You are a radiologist. Your task is to determine whether a given context sentence from radiology reports contains the answer to a provided question. Please respond only with 'entailment' or 'not entailment' without any additional explanations.

    Here are three examples:
    Context sentence: There is no pleural effusion or pneumothorax. Question: Are there signs of pneumothorax?    entailment
    Context sentence: In the region of the subsegmental branch in the right lower lobe there is a suggestion of filling defect that is of uncertain significance. Question: Is there any evidence of bleeding in the chest cavity?    not entailment
    Context sentence: No evidence of pneumothorax or other complications. Question: Do we see any trauma to the pluera due to Central venous line?    entailment

    Determine whether the given context sentence from radiology reports contains the answer to the provided question. Context sentence: {} Question: {}
    """.format(sentence, question)

    prompt = generate_prompt(instruction, input)
    # print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=128
    )

    output = tokenizer.decode(generation_output.sequences[0])
    return output.split("### Response:")[1].strip()

In [ ]:
sentence = "No evidence of pneumothorax or other complications."  # not entailment
question = "Is the central venous line in the location it was aimed at?"


# sentence = "IMPRESSION: Cardiomegaly with worsening CHF and bilateral pleural effusions  compared to [**2155-11-29**]."  # entailment
# question = "Is there any new abnormalities after thoracentesis?"

for i in range(5):
  print(evaluate(sentence, question))

not entailment
not entailment
not entailment
not entailment
not entailment


## run

In [ ]:
import time

def process_row(args):
    # time.sleep(1)
    index, row = args
    corrected_text = evaluate(row['sentence'], row['question']).replace('not ', 'not_').replace('.', '')
    return index, corrected_text


input_file = "/content/drive/MyDrive/CAID/Data/bio_text/RadQA/radqa_test_balanced.csv"
df = pd.read_csv(input_file)

# parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
# with concurrent.futures.ThreadPoolExecutor() as executor:
    results = dict(tqdm(executor.map(process_row, df.iterrows()), total=len(df)))

# same order as the input
df['predict'] = df.index.map(results.get)

output_file = "/content/drive/MyDrive/CAID/Data/bio_text/RadQA/predict_alpaca_native.csv"
df.to_csv(output_file, index=False)

100%|██████████| 920/920 [28:56<00:00,  1.89s/it]


## Eval

In [ ]:
predict = pd.read_csv(output_file)
predict['predict'] = predict['predict'].str.replace('2', 'entailment', regex=True)
# valid_values = ['entailment', 'not_entailment']
# predict['predict'] = predict['predict'].apply(lambda x: 'not_entailment' if x not in valid_values else x)
predict[['label', 'predict']].head(10)

In [ ]:
predict['predict'].unique()

array(['not_entailment', 'entailment'], dtype=object)

In [ ]:
import evaluate

In [ ]:
predict['label'] = predict['label'].replace('not_entailment', 0).replace('entailment', 1)
predict['predict'] = predict['predict'].replace('not_entailment', 0).replace('entailment', 1)

ACC = evaluate.load("accuracy")
P = evaluate.load("precision")
R = evaluate.load("recall")
F1 = evaluate.load("f1")
acc = ACC.compute(predictions=predict['predict'], references=predict['label'])
p = P.compute(predictions=predict['predict'], references=predict['label'])
r = R.compute(predictions=predict['predict'], references=predict['label'])
f1 = F1.compute(predictions=predict['predict'], references=predict['label'])
print('ACC: {:0.4f}'.format(acc['accuracy']))
print('P: {:0.4f}'.format(p['precision']))
print('R: {:0.4f}'.format(r['recall']))
print('F1: {:0.4f}'.format(f1['f1']))

ACC: 0.5435
P: 0.6695
R: 0.1717
F1: 0.2734


alpaca_native\
ACC: 0.5435\
P: 0.6695\
R: 0.1717\
F1: 0.2734\